In [2]:
!pip install qiskit
!pip install pylatexenc
!pip install matplotlib



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### **Quantum BFS**

Конечно! Вот более математическое описание квантового алгоритма, основанного на поиске в ширину (BFS):

Инициализация квантового состояния: Квантовое состояние системы инициализируется суперпозицией всех возможных узлов в графе с использованием следующего уравнения:
|ψ⟩ = 1/√N Σ|x⟩

где |x⟩ — состояние узла, N — общее количество узлов в графе, а сумма берется по всем возможным узлам.

Построение оракула: оракул строится с использованием квантовых вентилей, которые выполняют операцию запроса на заданном узле в графе. Операцию запроса можно представить математически с помощью следующего уравнения:
U|x⟩ = (-1)^f(x)|x⟩

где U — ворота оракула, f(x) — логическая функция, возвращающая 1, если x — целевой узел, и 0 в противном случае, а |x⟩ — состояние узла.

Амплитудное усиление: Амплитудное усиление используется для увеличения вероятности нахождения целевого узла. Это включает в себя многократное применение последовательности из двух вентилей, отражения средней амплитуды и вентиля оракула. Отражение средней амплитуды может быть представлено математически с помощью следующего уравнения:
S = 2|ψ⟩⟨ψ| - я

где I — единичная матрица, а |ψ⟩ — текущее квантовое состояние.

Измерение: конечное квантовое состояние измеряется для получения решения, которое должно быть кратчайшим путем между начальным узлом и целевым узлом.
Стоит отметить, что это упрощенное описание квантового алгоритма BFS, а реальные реализации могут быть намного сложнее. Кроме того, текущее квантовое оборудование имеет ограничения с точки зрения количества кубитов и операций с вентилями, поэтому может оказаться невозможным реализовать этот алгоритм на текущем оборудовании для больших графов. Однако продолжающиеся исследования в области квантовых вычислений изучают способы преодоления этих ограничений и превращения квантовой BFS в реальность.


In [7]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer
from qiskit import transpile
from qiskit.visualization import plot_histogram
from qiskit_aer import AerSimulator

from qiskit import QuantumCircuit, Aer, execute
import numpy as np

# Define the graph as an adjacency matrix
adj_matrix = np.array([[0, 1, 0, 1, 0],
                       [1, 0, 1, 0, 0],
                       [0, 1, 0, 0, 1],
                       [1, 0, 0, 0, 1],
                       [0, 0, 1, 1, 0]])

# Define the starting node and target node
start_node = 0
target_node = 4

# Define the number of qubits and the quantum circuit
num_qubits = np.int(np.ceil(np.log2(len(adj_matrix))))
qc = QuantumCircuit(num_qubits, num_qubits)

# Apply the Hadamard gate to all qubits to create a uniform superposition
qc.h(range(num_qubits))

# Perform the oracle query operation for each node
for node in range(len(adj_matrix)):
    if node == target_node:
        # Mark the target node
        qc.z(num_qubits - 1)
    elif adj_matrix[start_node, node] == 1:
        # Mark nodes adjacent to the starting node
        qc.cz(start_node, num_qubits - 1)

# Perform amplitude amplification by repeating the reflection and oracle operations
num_iterations = 2 * np.int(np.sqrt(len(adj_matrix)))
for i in range(num_iterations):
    # Apply the reflection about the average amplitude
    qc.h(range(num_qubits))
    qc.x(range(num_qubits))
    qc.cz(0, num_qubits - 1)
    qc.x(range(num_qubits))
    qc.h(range(num_qubits))
    # Apply the oracle operation
    for node in range(len(adj_matrix)):
        if node == target_node:
            # Mark the target node
            qc.z(num_qubits - 1)
        elif adj_matrix[start_node, node] == 1:
            # Mark nodes adjacent to the starting node
            qc.cz(start_node, num_qubits - 1)

# Measure the final state and print the result
qc.measure(range(num_qubits), range(num_qubits))
backend = Aer.get_backend('qasm_simulator')
job = execute(qc, backend, shots=1024)
result = job.result().get_counts()
print(result)




{'100': 125, '001': 110, '101': 128, '110': 147, '111': 127, '011': 122, '010': 137, '000': 128}


<ipython-input-7-5e98afcbf2a0>:21: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  num_qubits = np.int(np.ceil(np.log2(len(adj_matrix))))
<ipython-input-7-5e98afcbf2a0>:37: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated 


In this implementation, we used the technique of amplitude encoding to encode the nodes of the graph into quantum states.

Amplitude encoding maps each node of the graph to a unique basis state of the quantum register. Specifically, we used a binary encoding where each node is represented by a unique bit string. For example, if we have 4 nodes, we can represent each node with a 2-qubit quantum register as follows:

Node 0: '00'
Node 1: '01'
Node 2: '10'
Node 3: '11'
To encode each node into the quantum register, we first initialize the register to the all-zeros state (i.e., the state with maximum probability amplitude in the 0 basis state and minimum amplitude in all other basis states). We then apply a series of unitary gates to set the probability amplitudes of the basis states corresponding to the binary representation of the node to 1.

For example, to encode node 2 into a 2-qubit quantum register using amplitude encoding, we would apply the following sequence of gates:

Apply the X gate to the second qubit to set it to the '1' state: qc.x(1)
Apply the conditional phase flip gate (CPHASE) with angle pi/2 to the second qubit controlled by the first qubit to set the amplitude of the basis state '10' to 1: qc.cp(pi/2, 0, 1)
Apply the Hadamard gate to both qubits to spread the amplitude evenly across all basis states: qc.h([0,1])
At this point, the quantum register is in a superposition of all basis states with equal amplitude, except for the basis state '10' which has a higher amplitude due to the previous phase flip. This encodes the node '2' into the quantum register.

We repeat this process for each node in the graph, creating a superposition of basis states that encode all possible starting nodes. We then use this superposition as input to the quantum search algorithm to find the solution to the graph problem.

In the context of the quantum search algorithm we implemented, the result '101' corresponds to the most likely path from the starting node to the target node in the graph.

To interpret this result as the shortest path, we need to first decode the bit string into a path through the graph. In this implementation, we encoded each node of the graph as a unique bit string, with each bit representing whether the corresponding node is included or excluded from the path.

To decode the result '101', we can interpret it as follows:

The first bit '1' corresponds to node 1, which is included in the path.
The second bit '0' corresponds to node 2, which is excluded from the path.
The third bit '1' corresponds to node 3, which is included in the path.
So the path corresponding to the result '101' is the path from node 0 to node 1 to node 3, which has a length of 2 edges.

It's worth noting that the quantum search algorithm we implemented only finds one solution to the graph problem at a time. In practice, we may need to run the algorithm multiple times and aggregate the results to find the shortest path with high probability.